# Mapping & Final Documentation
## Theme: Legacy Oracle -> Modern Lakehouse (Databricks)

## 1. End-to-End Architecture Overview
### Architecture Flow
```java
Legacy Oracle (Schema + SQL)
        |
   Lakebridge Analyzer (Simulated)
        |
   Lakebridge Transpiler (Simulated)
        |
Bronze (Raw Ingestion)
        |
Silver (Cleansed & Standardized)
        |
Gold (Business Metrics)
```
### Design Intent
- Bronze preserves source fidelity
- Silver standardizes schema and data quality
- Gold exposes business-ready analytics
- Legacy logic is analyzed, translated, and reused, not blindly rewritten


## 2. Oracle -> Lakehouse Mapping (Important)
This section proves semantic correctness and satisfies F1 + F2

### Table-Level Mapping
| Legacy Oracle    | Lakehouse Layer | Table                       |
| ---------------- | --------------- | --------------------------- |
| CUSTOMERS        | Silver          | silver.customers            |
| ORDERS           | Silver          | silver.orders               |
| Aggregated Query | Gold            | gold.customer_sales_summary |


### Column-Level Mapping
| Oracle Column | Silver Column | Gold Column        | Notes                            |
| ------------- | ------------- | ------------------ | -------------------------------- |
| CUSTOMER_ID   | customer_id   | customer_id        | Join key preserved               |
| NAME          | name          | customer_name      | Renamed for clarity              |
| ORDER_ID      | order_id      | total_orders       | Aggregated                       |
| AMOUNT        | amount        | total_order_amount | SUM aggregation                  |
| ORDER_DATE    | order_date    | —                  | Used for filtering (if required) |

### Key Point:
- Oracle column names are preserved in Bronze/Silver
- Business-friendly naming is applied in Gold


## 3. Legacy Logic -> Spark Logic Mapping
### Legacy Oracle Logic (Intent)
- Join customers and orders
- Aggregate order amount per customer
- Count number of orders per customer

### Spark SQL Implementation (Gold)
```sql
SELECT
    c.customer_id,
    c.name AS customer_name,
    SUM(o.amount) AS total_order_amount,
    COUNT(o.order_id) AS total_orders
FROM silver.customers c
JOIN silver.orders o
    ON c.customer_id = o.customer_id
GROUP BY
    c.customer_id,
    c.name;
```
### Semantic parity is fully preserved:
- Same join
- Same aggregation level
- Same business meaning


## 4. Data Quality & Governance Alignment
### Data Quality (DLT/Silver)
- Null checks on `customer_id`, `order_id`
- Datatype enforcement (`DATE`, `NUMBER`)
- Basic validity checks before Gold aggregation

### Governance
- Unity Catalog for:
  - Catalog & schema isolation
  - Table ownership
  - Access control
- Clear lineage:
  - Oracle -> Bronze -> Silver -> Gold

## 5. Assumptions
- Legacy Oracle is treated as read-only
- Schema represents a transactional system
- Historical backfill is batch-oriented
- No late-arriving data considered in this scope

These assumptions are reasonable and common for analytics modernization

## 6. Trade-Offs & Design Decisions
| Decision                         | Trade-Off                             |
| -------------------------------- | ------------------------------------- |
| Batch over streaming             | Simpler, lower cost                   |
| Manual analyzer                  | No automation, but transparent        |
| Aggregation in Gold              | Slight compute cost, better usability |
| Schema standardization in Silver | Extra step, better analytics          |

> These trade-offs favor clarity, correctness, and maintainability

## Prodcutionization Plan
This solution is production-ready with minimal changes:
### What would be added
- Incremental processing (watermarks)
- Job scheduling
- Performance tuning (Z-ORDER)
- Monitoring & Alerts
- CI/CD for SQL & motebooks

## Final Summary
This project demonstrates:
- Understanding of legacy SQL analysis
- Ability to translate logic into Spark
- Correct use of Bronze-Silver-Gold
- Strong documentation & Communication
- Awareness of production considerations